In [1]:
import sys
from pathlib import Path

CWD = Path().resolve()
PROJECT_ROOT = CWD.parents[0]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("Project root:", PROJECT_ROOT)


Project root: /Users/juliantalou/Paddock/Asset/Valuation


In [3]:
pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 82.1 MB/s  0:00:00 102.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from src.parametric.fit_market_params import fit_params_for_car_market
from src.inference.msrp_extraction import upsert_msrp_for_market_car
from src.data.market_data import fetch_points_for_car_market
from src.data.marketcheck_client import fetch_marketcheck_points_for_car_market
from src.parametric.depreciation_function import value_universal
import numpy as np
import matplotlib.pyplot as plt


In [5]:
make = "BMW"
model = "Serie 3"
year = 2013
trim = "328i Sport"
country = "US"

In [6]:
msrp = upsert_msrp_for_market_car(make, model, year, trim, country)
msrp

APIError: {'message': "Could not find the table 'public.vehicles_market_nn_parameters' in the schema cache", 'code': 'PGRST205', 'hint': "Perhaps you meant the table 'public.vehicles_market_parameters'", 'details': None}

In [ ]:
result = fit_params_for_car_market(
    make=make,
    model=model,
    year=year,
    trim=trim,
    use_marketcheck=True,
)
result

In [ ]:
params = result["params"]

# Fetch data for plotting
data = fetch_points_for_car_market(make, model, year, trim)
t = data["t"]
m = data["m"]
v = data["v"]

# Predicted values
v_pred = value_universal(t, m, params)

plt.figure(figsize=(10,6))
plt.scatter(t, v, label="Observed (Supabase)", alpha=0.6)
plt.scatter(
    fetch_marketcheck_points_for_car_market(make, model, year, trim)["t"],
    fetch_marketcheck_points_for_car_market(make, model, year, trim)["v"],
    label="MarketCheck",
    alpha=0.5,
    color="orange"
)

# Smooth curve at fixed mileage (m=0)
t_grid = np.linspace(min(t), max(t), 200)
m0 = np.zeros_like(t_grid)
v_curve = value_universal(t_grid, m0, params)
plt.plot(t_grid, v_curve, label="Fitted market curve (m=0)", color="red")

plt.title(f"{make} {model} {trim} {year} — Market Curve Fit")
plt.xlabel("Age (years)")
plt.ylabel("Value (USD)")
plt.legend()
plt.grid(True)
plt.show()
